# 1 - Sequence to Sequence Learning with Neural Networks

In this series we'll be building a machine learning model to go from once sequence to another, using PyTorch and torchtext. This will be done on German to English translations, but the models can be applied to any problem that involves going from one sequence to another, such as summarization, i.e. going from a sequence to a shorter sequence in the same language.

In this first notebook, we'll start simple to understand the general concepts by implementing the model from the [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215) paper. 

## Introduction

The most common sequence-to-sequence (seq2seq) models are *encoder-decoder* models, which commonly use a *recurrent neural network* (RNN) to *encode* the source (input) sentence into a single vector. In this notebook, we'll refer to this single vector as a *context vector*. We can think of the context vector as being an abstract representation of the entire input sentence. This vector is then *decoded* by a second RNN which learns to output the target (output) sentence by generating it one word at a time.

![](assets/seq2seq1.png)

The above image shows an example translation. The input/source sentence, "guten morgen", is passed through the embedding layer (yellow) and then input into the encoder (green). We also append a *start of sequence* (`<sos>`) and *end of sequence* (`<eos>`) token to the start and end of sentence, respectively. At each time-step, the input to the encoder RNN is both the embedding, $e$, of the current word, $e(x_t)$, as well as the hidden state from the previous time-step, $h_{t-1}$, and the encoder RNN outputs a new hidden state $h_t$. <font color ='red'>We can think of the hidden state as a vector representation of the sentence so far. The RNN can be represented as a function of both of $e(x_t)$ and $h_{t-1}$:</font>

$$h_t = \text{EncoderRNN}(e(x_t), h_{t-1})$$

<font color ='red'>We're using the term RNN generally here, it could be any recurrent architecture, such as an *LSTM* (Long Short-Term Memory) or a *GRU* (Gated Recurrent Unit). </font>

Here, we have $X = \{x_1, x_2, ..., x_T\}$, where $x_1 = \text{<sos>}, x_2 = \text{guten}$, etc. The initial hidden state, $h_0$, is usually either initialized to zeros or a learned parameter.

Once the final word, $x_T$, has been passed into the RNN via the embedding layer, we use the final hidden state, $h_T$, as the context vector, i.e. $h_T = z$. This is a vector representation of the entire source sentence.

Now we have our context vector, $z$, we can start decoding it to get the output/target sentence, "good morning". Again, we append start and end of sequence tokens to the target sentence. At each time-step, the input to the decoder RNN (blue) is the embedding, $d$, of current word, $d(y_t)$, as well as the hidden state from the previous time-step, $s_{t-1}$, where the initial decoder hidden state, $s_0$, is the context vector, $s_0 = z = h_T$, i.e. the initial decoder hidden state is the final encoder hidden state. Thus, similar to the encoder, we can represent the decoder as:

$$s_t = \text{DecoderRNN}(d(y_t), s_{t-1})$$

Although the input/source embedding layer, $e$, and the output/target embedding layer, $d$, are both shown in yellow in the diagram they are two different embedding layers with their own parameters.

In the decoder, we need to go from the hidden state to an actual word, therefore at each time-step we use $s_t$ to predict (by passing it through a `Linear` layer, shown in purple) what we think is the next word in the sequence, $\hat{y}_t$. 

$$\hat{y}_t = f(s_t)$$

The words in the decoder are always generated one after another, with one per time-step. <font color ='red'>**We always use `<sos>` for the first input to the decoder, $y_1$, but for subsequent inputs, $y_{t>1}$, we will sometimes use the actual, ground truth next word in the sequence, $y_t$ and sometimes use the word predicted by our decoder, $\hat{y}_{t-1}$. This is called *teacher forcing*, see a bit more info about it [here]**</font>(https://machinelearningmastery.com/teacher-forcing-for-recurrent-neural-networks/). 

<font color ='red'>WWhen training/testing our model, we always know how many words are in our target sentence, so we stop generating words once we hit that many. During inference it is common to keep generating words until the model outputs an `<eos>` token or after a certain amount of words have been generated.</font>

Once we have our predicted target sentence, $\hat{Y} = \{ \hat{y}_1, \hat{y}_2, ..., \hat{y}_T \}$, we compare it against our actual target sentence, $Y = \{ y_1, y_2, ..., y_T \}$, to calculate our loss. We then use this loss to update all of the parameters in our model.

## Import Libraries



In [3]:
# code here

In [4]:
#from google.colab import drive
#drive.mount('/content/drive')

In [5]:
data_folder = Path('/home/harpreet/Insync/google_drive_shaannoor/Data/NLP')
project_folder = Path('/home/harpreet/Insync/google_drive_harpreet/Research/NLP/pytorch-seq2seq')
#data_folder = Path('/content/drive/MyDrive/Data/NLP')

In [ ]:
#!pip install -U spacy

# Set Seeds

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## Load Tokenized Data

In [6]:
# this was saved in 2a ---- we can ignore cells above  this
df_train = pd.read_pickle(project_folder/'df_train_en_de.pickel')
df_test =  pd.read_pickle(project_folder/'df_test_en_de.pickel')
df_valid = pd.read_pickle(project_folder/'df_valid_en_de.pickel')

In [7]:
import swifter
df_train['source_tokens_len'] = df_train['source_tokens'].swifter.apply(lambda x: len(x))

Pandas Apply:   0%|          | 0/29000 [00:00<?, ?it/s]

In [8]:
df_train['source_tokens_len'].mean()

12.713896551724138

In [9]:
df_train['source_tokens_len'].min()

1

In [10]:
df_train['source_tokens_len'].max()

46

In [ ]:
df_train

,source_tokens,target_tokens,source_tokens_reverse
0,"[zwei, junge, weiße, männer, sind, i, m, freie...","[two, young, ,, white, males, are, outside, ne...","[., büsche, vieler, nähe, der, in, freien, m, ..."
1,"[mehrere, männer, mit, schutzhelmen, bedienen,...","[several, men, in, hard, hats, are, operating,...","[., antriebsradsystem, ein, bedienen, schutzhe..."
2,"[ein, kleines, mädchen, klettert, in, ein, spi...","[a, little, girl, climbing, into, a, wooden, p...","[., holz, aus, spielhaus, ein, in, klettert, m..."
3,"[ein, mann, in, einem, blauen, hemd, steht, au...","[a, man, in, a, blue, shirt, is, standing, on,...","[., fenster, ein, putzt, und, leiter, einer, a..."
4,"[zwei, männer, stehen, am, herd, und, bereiten...","[two, men, are, at, the, stove, preparing, foo...","[., zu, essen, bereiten, und, herd, am, stehen..."
...,...,...,...
28995,"[., wand, verschnörkelten, einer, hinter, schr...","[a, woman, behind, a, scrolled, wall, is, writ...","[eine, frau, schreibt, hinter, einer, verschnö..."
28996,"[., kletterwand, einer, an, übt, bergsteiger, ...","[a, rock, climber, practices, on, a, rock, cli...","[ein, bergsteiger, übt, an, einer, kletterwand..."
28997,"[., hauses, einem, vor, straße, einer, auf, ar...","[two, male, construction, workers, are, workin...","[zwei, bauarbeiter, arbeiten, auf, einer, stra..."
28998,"[., fassade, einer, vor, wagen, einem, mit, ju...","[an, elderly, man, sits, outside, a, storefron...","[ein, älterer, mann, sitzt, mit, einem, jungen..."


In [ ]:
print(df_train['source_tokens_reverse'][0])

['.', 'büsche', 'vieler', 'nähe', 'der', 'in', 'freien', 'm', 'i', 'sind', 'männer', 'weiße', 'junge', 'zwei']


In [ ]:
df_test

,source_tokens,target_tokens,source_tokens_reverse
0,"[ein, mann, mit, einem, orangefarbenen, hut, ,...","[a, man, in, an, orange, hat, starring, at, so...","[., anstarrt, etwas, der, ,, hut, orangefarben..."
1,"[ein, boston, terrier, läuft, über, saftig, -,...","[a, boston, terrier, is, running, on, lush, gr...","[., zaun, weißen, einem, vor, gras, grünes, -,..."
2,"[ein, mädchen, in, einem, karateanzug, bricht,...","[a, girl, in, karate, uniform, breaking, a, st...","[., tritt, einem, mit, stock, einen, bricht, k..."
3,"[fünf, leute, in, winterjacken, und, mit, helm...","[five, people, wearing, winter, jackets, and, ...","[., hintergrund, m, i, schneemobilen, mit, sch..."
4,"[leute, reparieren, das, dach, eines, hauses, .]","[people, are, fixing, the, roof, of, a, house, .]","[., hauses, eines, dach, das, reparieren, leute]"
...,...,...,...
995,"[., stehen, herum, sie, um, personen, andere, ...","[marathon, runners, are, racing, on, a, city, ...","[marathonläufer, laufen, auf, einer, städtisch..."
996,"[., fahrradfahren, beim, sonnenhut, einen, trä...","[asian, woman, wearing, a, sunhat, while, ridi...","[asiatische, frau, trägt, einen, sonnenhut, be..."
997,"[., bäumen, zwei, bei, boden, dem, auf, spiele...","[some, children, are, outside, playing, in, th...","[ein, paar, kinder, sind, i, m, freien, und, s..."
998,"[., videospiel, ein, spielt, mann, älterer, ein]","[an, older, man, is, playing, a, video, arcade...","[ein, älterer, mann, spielt, ein, videospiel, .]"


In [ ]:
df_valid

,source_tokens,target_tokens,source_tokens_reverse
0,"[eine, gruppe, von, männern, lädt, baumwolle, ...","[a, group, of, men, are, loading, cotton, onto...","[lastwagen, einen, auf, baumwolle, lädt, männe..."
1,"[ein, mann, schläft, in, einem, grünen, raum, ...","[a, man, sleeping, in, a, green, room, on, a, ...","[., sofa, einem, auf, raum, grünen, einem, in,..."
2,"[ein, junge, mit, kopfhörern, sitzt, auf, den,...","[a, boy, wearing, headphones, sits, on, a, wom...","[., frau, einer, schultern, den, auf, sitzt, k..."
3,"[zwei, männer, bauen, eine, blaue, eisfischerh...","[two, men, setting, up, a, blue, ice, fishing,...","[auf, see, zugefrorenen, einem, auf, eisfische..."
4,"[ein, mann, mit, beginnender, glatze, ,, der, ...","[a, balding, man, wearing, a, red, life, jacke...","[., boot, kleinen, einem, in, sitzt, ,, trägt,..."
...,...,...,...
1009,"[., her, zuckerwatte, mann, ein, stellt, jahrm...","[at, some, sort, of, carnival, ,, a, man, is, ...","[bei, einer, art, jahrmarkt, stellt, ein, mann..."
1010,"[., bus, einem, vor, steht, polizisten, von, g...","[a, bunch, of, police, officers, are, standing...","[eine, gruppe, von, polizisten, steht, vor, ei..."
1011,"[., hindurch, brillengläser, ihre, durch, blic...","[a, elderly, white, -, haired, woman, is, look...","[eine, ältere, weißhaarige, frau, sieht, in, i..."
1012,"[., freien, m, i, telefonzellen, an, stehen, m...","[two, men, are, standing, at, telephone, booth...","[zwei, männer, stehen, an, telefonzellen, i, m..."


## Build Vocab

In [ ]:
def create_vocab(text, min_freq, specials):
    # code here


In [ ]:
source_vocab = # code here

In [ ]:
len(source_vocab)

7874

In [ ]:
pd.DataFrame(source_vocab.get_stoi().items(), columns=['tokens', 'index']).sort_values(by = ['index'])

,tokens,index
4984,<unk>,0
3312,<BOS>,1
3863,<EOS>,2
4540,<PAD>,3
3777,.,4
...,...,...
2,geländefahrzeugs,7869
1,sonnentag,7870
0,lederoberteil,7871
2376,reisfeld,7872


In [ ]:
# check index of unknown word - it should be zero
source_vocab['abracdabra']

0

In [ ]:
target_vocab = create_vocab(df_train['target_tokens'], 2, ['<unk>', '<BOS>', '<EOS>', '<PAD>'])

In [ ]:
len(target_vocab)

5893

# Create Dataset and Dataloader

In [ ]:
class EngGerman(Dataset):
    
    # code here

     
    

In [ ]:
trainset = # code here
testset =  # code here
validset = # code here

In [ ]:
trainset.__getitem__(0)

(['.',
  'büsche',
  'vieler',
  'nähe',
  'der',
  'in',
  'freien',
  'm',
  'i',
  'sind',
  'männer',
  'weiße',
  'junge',
  'zwei'],
 ['two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.'])

In [ ]:
len(trainset), len(testset), len(validset)

(29000, 1000, 1014)

In [ ]:
# return a list for index of tokens in following order [<BOS> ,text tokens ,<EOS>]
def text_transform (my_vocab, text):
 

In [ ]:
text = trainset.__getitem__(12)[1]
print(text)
text_transform(target_vocab, text)

['a', 'black', 'dog', 'and', 'a', 'spotted', 'dog', 'are', 'fighting']


tensor([ 1, 21, 90, 91, 92, 21, 93, 91,  9, 94,  2])

In [ ]:
# return padded sequences for a batch
def collate_batch(batch):


In [ ]:
batch_size = 2

train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, 
                              collate_fn=collate_batch)

In [ ]:
for source, target in train_loader:
  print(source)
  break

tensor([[   1,    1],
        [  18,   52],
        [3864,   73],
        [ 834,   21],
        [ 205,  249],
        [ 141,  253],
        [ 562,   58],
        [  50,    0],
        [  49,  413],
        [ 400,   41],
        [ 194,   90],
        [   9, 7538],
        [   8,    4],
        [ 657,    2],
        [ 576,    3],
        [   4,    3],
        [   2,    3]])


In [ ]:
BATCH_SIZE = 128

train_loader =  # code here
valid_loader =  # code here
test_loader =   # code here

## Building the Seq2Seq Model

We'll be building our model in three parts. The encoder, the decoder and a seq2seq model that encapsulates the encoder and decoder and will provide a way to interface with each.

### Encoder

First, the encoder, a 2 layer LSTM. The paper we are implementing uses a 4-layer LSTM, but in the interest of training time we cut this down to 2-layers. The concept of multi-layer RNNs is easy to expand from 2 to 4 layers. 

For a multi-layer RNN, the input sentence, $X$, after being embedded goes into the first (bottom) layer of the RNN and hidden states, $H=\{h_1, h_2, ..., h_T\}$, output by this layer are used as inputs to the RNN in the layer above. Thus, representing each layer with a superscript, the hidden states in the first layer are given by:

$$h_t^1 = \text{EncoderRNN}^1(e(x_t), h_{t-1}^1)$$

The hidden states in the second layer are given by:

$$h_t^2 = \text{EncoderRNN}^2(h_t^1, h_{t-1}^2)$$

Using a multi-layer RNN also means we'll also need an initial hidden state as input per layer, $h_0^l$, and we will also output a context vector per layer, $z^l$.

Without going into too much detail about LSTMs (see [this](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) blog post to learn more about them), all we need to know is that they're a type of RNN which instead of just taking in a hidden state and returning a new hidden state per time-step, also take in and return a *cell state*, $c_t$, per time-step.

$$\begin{align*}
h_t &= \text{RNN}(e(x_t), h_{t-1})\\
(h_t, c_t) &= \text{LSTM}(e(x_t), h_{t-1}, c_{t-1})
\end{align*}$$

We can just think of $c_t$ as another type of hidden state. Similar to $h_0^l$, $c_0^l$ will be initialized to a tensor of all zeros. Also, our context vector will now be both the final hidden state and the final cell state, i.e. $z^l = (h_T^l, c_T^l)$.

Extending our multi-layer equations to LSTMs, we get:

$$\begin{align*}
(h_t^1, c_t^1) &= \text{EncoderLSTM}^1(e(x_t), (h_{t-1}^1, c_{t-1}^1))\\
(h_t^2, c_t^2) &= \text{EncoderLSTM}^2(h_t^1, (h_{t-1}^2, c_{t-1}^2))
\end{align*}$$

Note how only our hidden state from the first layer is passed as input to the second layer, and not the cell state.

So our encoder looks something like this: 

![](assets/seq2seq2.png)

We create this in code by making an `Encoder` module, which requires we inherit from `torch.nn.Module` and use the `super().__init__()` as some boilerplate code. The encoder takes the following arguments:
- `input_dim` is the size/dimensionality of the one-hot vectors that will be input to the encoder. This is equal to the input (source) vocabulary size.
- `emb_dim` is the dimensionality of the embedding layer. This layer converts the one-hot vectors into dense vectors with `emb_dim` dimensions. 
- `hid_dim` is the dimensionality of the hidden and cell states.
- `n_layers` is the number of layers in the RNN.
- `dropout` is the amount of dropout to use. This is a regularization parameter to prevent overfitting. Check out [this](https://www.coursera.org/lecture/deep-neural-network/understanding-dropout-YaGbR) for more details about dropout.

We aren't going to discuss the embedding layer in detail during these tutorials. All we need to know is that there is a step before the words - technically, the indexes of the words - are passed into the RNN, where the words are transformed into vectors. To read more about word embeddings, check these articles: [1](https://monkeylearn.com/blog/word-embeddings-transform-text-numbers/), [2](http://p.migdal.pl/2017/01/06/king-man-woman-queen-why.html), [3](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/), [4](http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/). 

The embedding layer is created using `nn.Embedding`, the LSTM with `nn.LSTM` and a dropout layer with `nn.Dropout`. Check the PyTorch [documentation](https://pytorch.org/docs/stable/nn.html) for more about these.

One thing to note is that the `dropout` argument to the LSTM is how much dropout to apply between the layers of a multi-layer RNN, i.e. between the hidden states output from layer $l$ and those same hidden states being used for the input of layer $l+1$.

In the `forward` method, we pass in the source sentence, $X$, which is converted into dense vectors using the `embedding` layer, and then dropout is applied. These embeddings are then passed into the RNN. As we pass a whole sequence to the RNN, it will automatically do the recurrent calculation of the hidden states over the whole sequence for us! Notice that we do not pass an initial hidden or cell state to the RNN. This is because, as noted in the [documentation](https://pytorch.org/docs/stable/nn.html#torch.nn.LSTM), that if no hidden/cell state is passed to the RNN, it will automatically create an initial hidden/cell state as a tensor of all zeros. 

The RNN returns: `outputs` (the top-layer hidden state for each time-step), `hidden` (the final hidden state for each layer, $h_T$, stacked on top of each other) and `cell` (the final cell state for each layer, $c_T$, stacked on top of each other).

As we only need the final hidden and cell states (to make our context vector), `forward` only returns `hidden` and `cell`. 

The sizes of each of the tensors is left as comments in the code. In this implementation `n_directions` will always be 1, however note that bidirectional RNNs (covered in tutorial 3) will have `n_directions` as 2.

In [ ]:
class Encoder(nn.Module):
    


### Decoder

Next, we'll build our decoder, which will also be a 2-layer (4 in the paper) LSTM.

![](assets/seq2seq3.png)

The `Decoder` class does a single step of decoding, i.e. it ouputs single token per time-step. The first layer will receive a hidden and cell state from the previous time-step, $(s_{t-1}^1, c_{t-1}^1)$, and feeds it through the LSTM with the current embedded token, $y_t$, to produce a new hidden and cell state, $(s_t^1, c_t^1)$. The subsequent layers will use the hidden state from the layer below, $s_t^{l-1}$, and the previous hidden and cell states from their layer, $(s_{t-1}^l, c_{t-1}^l)$. This provides equations very similar to those in the encoder.

$$\begin{align*}
(s_t^1, c_t^1) = \text{DecoderLSTM}^1(d(y_t), (s_{t-1}^1, c_{t-1}^1))\\
(s_t^2, c_t^2) = \text{DecoderLSTM}^2(s_t^1, (s_{t-1}^2, c_{t-1}^2))
\end{align*}$$

Remember that the initial hidden and cell states to our decoder are our context vectors, which are the final hidden and cell states of our encoder from the same layer, i.e. $(s_0^l,c_0^l)=z^l=(h_T^l,c_T^l)$.

We then pass the hidden state from the top layer of the RNN, $s_t^L$, through a linear layer, $f$, to make a prediction of what the next token in the target (output) sequence should be, $\hat{y}_{t+1}$. 

$$\hat{y}_{t+1} = f(s_t^L)$$

The arguments and initialization are similar to the `Encoder` class, except we now have an `output_dim` which is the size of the vocabulary for the output/target. There is also the addition of the `Linear` layer, used to make the predictions from the top layer hidden state.

Within the `forward` method, we accept a batch of input tokens, previous hidden states and previous cell states. As we are only decoding one token at a time, the input tokens will always have a sequence length of 1. We `unsqueeze` the input tokens to add a sentence length dimension of 1. Then, similar to the encoder, we pass through an embedding layer and apply dropout. This batch of embedded tokens is then passed into the RNN with the previous hidden and cell states. This produces an `output` (hidden state from the top layer of the RNN), a new `hidden` state (one for each layer, stacked on top of each other) and a new `cell` state (also one per layer, stacked on top of each other). We then pass the `output` (after getting rid of the sentence length dimension) through the linear layer to receive our `prediction`. We then return the `prediction`, the new `hidden` state and the new `cell` state.

**Note**: as we always have a sequence length of 1, we could use `nn.LSTMCell`, instead of `nn.LSTM`, as it is designed to handle a batch of inputs that aren't necessarily in a sequence. `nn.LSTMCell` is just a single cell and `nn.LSTM` is a wrapper around potentially multiple cells. Using the `nn.LSTMCell` in this case would mean we don't have to `unsqueeze` to add a fake sequence length dimension, but we would need one `nn.LSTMCell` per layer in the decoder and to ensure each `nn.LSTMCell` receives the correct initial hidden state from the encoder. All of this makes the code less concise - hence the decision to stick with the regular `nn.LSTM`.

In [ ]:
class Decoder(nn.Module):


### Seq2Seq

For the final part of the implemenetation, we'll implement the seq2seq model. This will handle: 
- receiving the input/source sentence
- using the encoder to produce the context vectors 
- using the decoder to produce the predicted output/target sentence

Our full model will look like this:

![](assets/seq2seq4.png)

The `Seq2Seq` model takes in an `Encoder`, `Decoder`, and a `device` (used to place tensors on the GPU, if it exists).

For this implementation, we have to ensure that the number of layers and the hidden (and cell) dimensions are equal in the `Encoder` and `Decoder`. This is not always the case, we do not necessarily need the same number of layers or the same hidden dimension sizes in a sequence-to-sequence model. However, if we did something like having a different number of layers then we would need to make decisions about how this is handled. For example, if our encoder has 2 layers and our decoder only has 1, how is this handled? Do we average the two context vectors output by the decoder? Do we pass both through a linear layer? Do we only use the context vector from the highest layer? Etc.

Our `forward` method takes the source sentence, target sentence and a teacher-forcing ratio. The teacher forcing ratio is used when training our model. When decoding, at each time-step we will predict what the next token in the target sequence will be from the previous tokens decoded, $\hat{y}_{t+1}=f(s_t^L)$. With probability equal to the teaching forcing ratio (`teacher_forcing_ratio`) we will use the actual ground-truth next token in the sequence as the input to the decoder during the next time-step. However, with probability `1 - teacher_forcing_ratio`, we will use the token that the model predicted as the next input to the model, even if it doesn't match the actual next token in the sequence.  

The first thing we do in the `forward` method is to create an `outputs` tensor that will store all of our predictions, $\hat{Y}$.

We then feed the input/source sentence, `src`, into the encoder and receive out final hidden and cell states.

The first input to the decoder is the start of sequence (`<sos>`) token. As our `trg` tensor already has the `<sos>` token appended (all the way back when we defined the `init_token` in our `TRG` field) we get our $y_1$ by slicing into it. We know how long our target sentences should be (`max_len`), so we loop that many times. The last token input into the decoder is the one **before** the `<eos>` token - the `<eos>` token is never input into the decoder. 

During each iteration of the loop, we:
- pass the input, previous hidden and previous cell states ($y_t, s_{t-1}, c_{t-1}$) into the decoder
- receive a prediction, next hidden state and next cell state ($\hat{y}_{t+1}, s_{t}, c_{t}$) from the decoder
- place our prediction, $\hat{y}_{t+1}$/`output` in our tensor of predictions, $\hat{Y}$/`outputs`
- decide if we are going to "teacher force" or not
    - if we do, the next `input` is the ground-truth next token in the sequence, $y_{t+1}$/`trg[t]`
    - if we don't, the next `input` is the predicted next token in the sequence, $\hat{y}_{t+1}$/`top1`, which we get by doing an `argmax` over the output tensor
    
Once we've made all of our predictions, we return our tensor full of predictions, $\hat{Y}$/`outputs`.

**Note**: our decoder loop starts at 1, not 0. This means the 0th element of our `outputs` tensor remains all zeros. So our `trg` and `outputs` look something like:

$$\begin{align*}
\text{trg} = [<sos>, &y_1, y_2, y_3, <eos>]\\
\text{outputs} = [0, &\hat{y}_1, \hat{y}_2, \hat{y}_3, <eos>]
\end{align*}$$

Later on when we calculate the loss, we cut off the first element of each tensor to get:

$$\begin{align*}
\text{trg} = [&y_1, y_2, y_3, <eos>]\\
\text{outputs} = [&\hat{y}_1, \hat{y}_2, \hat{y}_3, <eos>]
\end{align*}$$

In [2]:
class Seq2Seq(nn.Module):


NameError: ignored

# Training the Seq2Seq Model

Now we have our model implemented, we can begin training it. 

First, we'll initialize our model. As mentioned before, the input and output dimensions are defined by the size of the vocabulary. The embedding dimesions and dropout for the encoder and decoder can be different, but the number of layers and the size of the hidden/cell states must be the same. 

We then define the encoder, decoder and then our Seq2Seq model, which we place on the `device`.

In [ ]:
INPUT_DIM = len(source_vocab)
OUTPUT_DIM = len(target_vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

Next up is initializing the weights of our model. In the paper they state they initialize all weights from a uniform distribution between -0.08 and +0.08, i.e. $\mathcal{U}(-0.08, 0.08)$.

We initialize weights in PyTorch by creating a function which we `apply` to our model. When using `apply`, the `init_weights` function will be called on every module and sub-module within our model. For each module we loop through all of the parameters and sample them from a uniform distribution with `nn.init.uniform_`.

In [ ]:
def init_weights(m):
    # code here

        
#model.apply(init_weights)

We also define a function that will calculate the number of trainable parameters in the model.

In [ ]:
def count_parameters(model):
    # code here

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 13,903,877 trainable parameters


We define our optimizer, which we use to update our parameters in the training loop. Check out [this](http://ruder.io/optimizing-gradient-descent/) post for information about different optimizers. Here, we'll use Adam.

In [ ]:
optimizer = # code here

Next, we define our loss function. The `CrossEntropyLoss` function calculates both the log softmax as well as the negative log-likelihood of our predictions. 

Our loss function calculates the average loss per token, however by passing the index of the `<pad>` token as the `ignore_index` argument we ignore the loss whenever the target token is a padding token. 

In [ ]:
TRG_PAD_IDX = target_vocab['<PAD>']
TRG_PAD_IDX

3

In [ ]:
criterion = # code here

Next, we'll define our training loop. 

First, we'll set the model into "training mode" with `model.train()`. This will turn on dropout (and batch normalization, which we aren't using) and then iterate through our data iterator.

As stated before, our decoder loop starts at 1, not 0. This means the 0th element of our `outputs` tensor remains all zeros. So our `trg` and `outputs` look something like:

$$\begin{align*}
\text{trg} = [<sos>, &y_1, y_2, y_3, <eos>]\\
\text{outputs} = [0, &\hat{y}_1, \hat{y}_2, \hat{y}_3, <eos>]
\end{align*}$$

Here, when we calculate the loss, we cut off the first element of each tensor to get:

$$\begin{align*}
\text{trg} = [&y_1, y_2, y_3, <eos>]\\
\text{outputs} = [&\hat{y}_1, \hat{y}_2, \hat{y}_3, <eos>]
\end{align*}$$

At each iteration:
- get the source and target sentences from the batch, $X$ and $Y$
- zero the gradients calculated from the last batch
- feed the source and target into the model to get the output, $\hat{Y}$
- as the loss function only works on 2d inputs with 1d targets we need to flatten each of them with `.view`
    - we slice off the first column of the output and target tensors as mentioned above
- calculate the gradients with `loss.backward()`
- clip the gradients to prevent them from exploding (a common issue in RNNs)
- update the parameters of our model by doing an optimizer step
- sum the loss value to a running total

Finally, we return the loss that is averaged over all batches.

In [ ]:
def train(model, iterator, optimizer, criterion, clip):


Our evaluation loop is similar to our training loop, however as we aren't updating any parameters we don't need to pass an optimizer or a clip value.

We must remember to set the model to evaluation mode with `model.eval()`. This will turn off dropout (and batch normalization, if used).

We use the `with torch.no_grad()` block to ensure no gradients are calculated within the block. This reduces memory consumption and speeds things up. 

The iteration loop is similar (without the parameter updates), however we must ensure we turn teacher forcing off for evaluation. This will cause the model to only use it's own predictions to make further predictions within a sentence, which mirrors how it would be used in deployment.

In [ ]:
def evaluate(model, iterator, criterion):
    pass
    


Next, we'll create a function that we'll use to tell us how long an epoch takes.

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

We can finally start training our model!

At each epoch, we'll be checking if our model has achieved the best validation loss so far. If it has, we'll update our best validation loss and save the parameters of our model (called `state_dict` in PyTorch). Then, when we come to test our model, we'll use the saved parameters used to achieve the best validation loss. 

We'll be printing out both the loss and the perplexity at each epoch. It is easier to see a change in perplexity than a change in loss as the numbers are much bigger.

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    # code here
    


Epoch: 01 | Time: 0m 23s
	Train Loss: 5.059 | Train PPL: 157.476
	 Val. Loss: 4.992 |  Val. PPL: 147.214
Epoch: 02 | Time: 0m 23s
	Train Loss: 4.501 | Train PPL:  90.148
	 Val. Loss: 4.654 |  Val. PPL: 105.041
Epoch: 03 | Time: 0m 23s
	Train Loss: 4.161 | Train PPL:  64.152
	 Val. Loss: 4.442 |  Val. PPL:  84.967
Epoch: 04 | Time: 0m 24s
	Train Loss: 3.905 | Train PPL:  49.657
	 Val. Loss: 4.224 |  Val. PPL:  68.273
Epoch: 05 | Time: 0m 24s
	Train Loss: 3.703 | Train PPL:  40.570
	 Val. Loss: 4.139 |  Val. PPL:  62.727
Epoch: 06 | Time: 0m 24s
	Train Loss: 3.545 | Train PPL:  34.627
	 Val. Loss: 4.005 |  Val. PPL:  54.860
Epoch: 07 | Time: 0m 24s
	Train Loss: 3.416 | Train PPL:  30.438
	 Val. Loss: 3.974 |  Val. PPL:  53.214
Epoch: 08 | Time: 0m 24s
	Train Loss: 3.282 | Train PPL:  26.639
	 Val. Loss: 3.946 |  Val. PPL:  51.727
Epoch: 09 | Time: 0m 24s
	Train Loss: 3.181 | Train PPL:  24.059
	 Val. Loss: 3.797 |  Val. PPL:  44.548
Epoch: 10 | Time: 0m 24s
	Train Loss: 3.077 | Train PPL

We'll load the parameters (`state_dict`) that gave our model the best validation loss and run it the model on the test set.

In [ ]:
model.load_state_dict(torch.load('tut1-model_1a.pt'))

test_loss = evaluate(model, test_loader, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.800 | Test PPL:  44.717 |


In the following notebook we'll implement a model that achieves improved test perplexity, but only uses a single layer in the encoder and the decoder.